In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git
!git clone https://github.com/shansen5/CA_Prisons.git
print( 'Done!' )

In [ ]:
%run ECHO_modules/DataSet.py
%run ECHO_modules/make_data_sets.py

### 1. Read the local spreadsheet with the carceral facilities

In [ ]:
carc_data  = pd.read_csv( "CA_Prisons/reformatted_fixed_GG_manual_coding_hifld_prisons_to_FRS.csv", encoding='iso-8859-1',
                 dtype={"Total_FRS_IDS": "Int64"} )
registry_ids = carc_data['Total_FRS_IDS'].dropna().unique()
print( "Number of records in CSV = {}, number of non-NA ids = {}".format( len(carc_data), len(registry_ids)))


### 2. Look up the facilities in ECHO_EXPORTER.  The records found will be written to ECHO_EXPORTER-CA_Carceral.csv.

If you have run this notebook before and have the ECHO_EXPORTER-CA_Carceral.csv file
already generated, you can use the following cell to read it directly and
save some time compared to the full database query.  If you don't have the
CSV file, skip to the next cell to get the data from the database.

In [ ]:
from os.path import exists

if ( exists("CA_Prisons/ECHO_EXPORTER-CA_Carceral.csv" )):
    echo_data = pd.read_csv( "CA_Prisons/ECHO_EXPORTER-CA_Carceral.csv" )
    echo_data.set_index( 'REGISTRY_ID', inplace=True)
else:
    print( "The file doesn't exist" )

Run this cell in any case.

In [ ]:
id_list = []
for id in registry_ids:
    id_list.append( id )

If you don't have the ECHO_EXPORTER-CA_Carceral.csv file from a previous run, use this cell.  It will take some time.

In [ ]:

    
ds = DataSet( name="exporter", base_table="ECHO_EXPORTER",
            table_name="ECHO_EXPORTER", echo_type=None, idx_field="REGISTRY_ID" )
echo_data = ds.get_data_by_ee_ids( id_list )

echo_data.to_csv( "ECHO_EXPORTER-CA_Carceral.csv")

### 3. Look up the facilities in the program data sets.  The records found will be written to CSV files for each data set.

In [ ]:
data_set_list = ['RCRA Violations', 'RCRA Inspections', 'RCRA Penalties',
                 'CAA Violations', 'CAA Inspections', 'CAA Penalties', 
                 'Greenhouse Gas Emissions', 'Toxic Releases',
                 'CWA Violations', 'CWA Inspections', 'CWA Penalties',
                 'SDWA Violations', 'SDWA Serious Violators', 'SDWA Site Visits',
                 'SDWA Enforcements', ] 
# You can run an individual data set, or just a few, by un-commenting and
# editing the following
# data_set_list = ['CWA Violations']
data_sets = make_data_sets( data_set_list )

last_flag = ''
pgm_ids = None
for pgm_name,ds in data_sets.items():
    print( pgm_name )
    if ( ds.echo_type == 'SDWA' ):
        echo_flag = 'SDWIS_FLAG'
    else:
        echo_flag = ds.echo_type + '_FLAG'
    # If the flag hasn't changed, the pgm_ids will still be the same.
    # Getting the pgm_ids from the echo_data's REGISTRY_ID takes a
    # long time, so we skip it if we can.
    if ( last_flag != echo_flag ):
        r_ids = echo_data[ echo_data[ echo_flag ] == 'Y' ].index
        pgm_ids = ds.get_pgm_ids( r_ids )
        last_flag = echo_flag
    program_data = ds.get_data_by_pgm_ids( pgm_ids )
    if ( program_data is not None):
        program_data.to_csv( pgm_name+'.csv' )

Parse out REGISTRY_IDs in order to query ICIS federal enforcement and compliance table

In [ ]:
len(id_list)

In [ ]:
ids = ""
for id in id_list[1:10]: # test on just 20 REGISTRY_IDs
    ids+="'"+str(id)+"',"
ids=ids[:-1] #remove trailing comma
ids=ids+",'110013272680'" #Texas Department of Corrections facility (for testing purposes)
ids

Get case numbers based on REGISTRY_IDs

In [ ]:
facilities = None
try:
  #sql = 'select * from "CASE_FACILITIES" where "REGISTRY_ID" = \'110013272680\'' #Texas Department of Corrections facility (for testing purposes)
  sql = 'select * from "CASE_FACILITIES" where "REGISTRY_ID" in ('+ids+')'
  url= 'http://portal.gss.stonybrook.edu/echoepa/index.php?query=' # Old server: 'http://apps.tlt.stonybrook.edu/echoepa/index2.php?query='
  data_location=url+urllib.parse.quote_plus(sql) + '&pg'
  print(sql) # For debugging
  print(data_location)
  facilities = pd.read_csv(data_location)
except pd.errors.EmptyDataError:
  print("No facilities match!")
facilities

Use case numbers to get enforcement action details

In [ ]:
#separate out case numbers in order to query the CASE_ENFORCEMENTS table
cases_list = list(facilities["CASE_NUMBER"].unique())
cases = ""
for case in cases_list:
  cases+="'"+case+"',"
cases=cases[:-1] #remove trailing comma
actions = None

#query the CASE_ENFORCEMENTS table
try:
  sql = 'select * from "CASE_ENFORCEMENTS" where "CASE_NUMBER" in ('+cases+')'
  url= 'http://portal.gss.stonybrook.edu/echoepa/index.php?query=' # Old server: 'http://apps.tlt.stonybrook.edu/echoepa/index2.php?query='
  data_location=url+urllib.parse.quote_plus(sql) + '&pg'
  print(sql) # For debugging
  print(data_location)
  actions = pd.read_csv(data_location)
except pd.errors.EmptyDataError:
  print("No actions found!")
actions

actions.to_csv('additional_enforcements.csv' )

In [ ]:
""